In [2]:
import neurora.rsa_plot

'''
虽然这是不经济的，但是还是搬用这个方法试一试，这是你现在手里能找到的唯一的方法了。
'''

'\n虽然这是不经济的，但是还是搬用这个方法试一试，这是你现在手里能找到的唯一的方法了。\n'

In [3]:
import os
import numpy as np
import mne
from mne.event import define_target_events
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from neurora import rsa_plot
from neurora.rdm_cal import eegRDM
from neurora.rdm_corr import rdm_correlation_spearman
from neurora.corr_cal_by_rdm import rdms_corr
from neurora.rsa_plot import plot_rdm
from neurora.rsa_plot import plot_corrs_by_time
from scipy.interpolate import interp1d
from neurora.rsa_plot import plot_corrs_hotmap
# from window_slider import Slider
from neurora.rsa_plot import plot_rdm, plot_corrs_by_time, plot_nps_hotmap, plot_corrs_hotmap,plot_tbyt_diff_decoding_acc

from typing import Dict, Tuple, List

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import signal
from nilearn import plotting, datasets, surface
import nibabel as nib
from neurora.stuff import get_affine, correct_by_threshold, get_bg_ch2, get_bg_ch2bet
from scipy import stats

1.3637687881961045


In [4]:
%matplotlib qt

In [5]:
"""
因为每个被试都要单独保持生成两个dissimilarity矩阵，
所以不能像原来那样粗暴地复制了，不划算，需要改写函数。
参数有哪些呢：
1 被试编号
全局变量给一个list
"""
#'用来存放dissimilarity（T1分散 VS T2分散） '
LIST_spaced = []
#'用来存放dissimilarity（T1分散 VS T2集中） '
LIST_massed = []

In [6]:
def eegdataOfOneField(channel_fieldn, epoch1, epoch2):
    """
    作者：LYW

    功能：从全脑数据中拿出来某个区域的数据

    变量：把这个field对应的channel组成的string list传进来

    返回值：crop好的eegdata

    """
    epock1_fieldn = epoch1.copy().pick_channels(channel_fieldn)  # 加copy是因为会在自身修改。
    epock2_fieldn = epoch2.copy().pick_channels(channel_fieldn)
    num_chan_picked = len(channel_fieldn)
    '''进行这两个条件之间STPS的计算'''
    # shape of eegdate: [n_cons, n_subs, n_chls, n_ts]
    eegdata = np.zeros([2, 1, num_chan_picked, 501], dtype=np.float32)
    # 把T1 T2合并起来，但是是建立一个新的维度：conditions，并不是直接拼接到一起。
    # 所以事先定好结构，然后赋值进去
    eegdata[0] = epock1_fieldn.data
    eegdata[1] = epock2_fieldn.data
    # 为了使用RSA函数，增加一个维度trial维度，两种condition下都只有一个trail（evoked）
    eegdata = np.reshape(eegdata, [2, 1, 1, num_chan_picked, 501])

    return eegdata

# 这个函数就是针对100ms的时间窗的

def OneLineOfSTPS(eegdata):
    """
    :arg:把evoked数据根据时间切块，每一个时间块都有对应的一个RSA，然后取RSA矩阵中的cell，作为这行STPS的值。
    :param eegdata: 2个条件下的eegdata,是
    :return: 只是十个值组成的一行的STPS
    """
    # 这里450是因为减掉了最前面的0.1基线部分。+
    OneLineOf_STPS = np.zeros([1, 450], dtype=np.float32)
    m = 0
    for m in range(450):
        n = m + 1
        ndarray = eegdata
        #用这个函数间接地求了矩阵之间的 差异值才对 Representational Dissimilarity Matrix
        #这里用的是time_win是默认值？
        #这里用的eegRDM最简单按的情况，相当于只是使用了计算 dissimilarity的部分。
        #n=0的时候，0到50，对应就是0ms到100ms的时间窗口。第一个算出来，应该放在-0.1s位置上。
        rdm = eegRDM(ndarray[:, :, :, :, n:n + 50],sub_opt=0)

        #50是和100ms的窗口保持一致。
        # rdm = eegRDM(ndarray[:, :, :, :, n:n + 50], time_opt=1, time_win=50)
        OneLineOf_STPS[0, m] = rdm[0, 1]
    return OneLineOf_STPS

In [7]:
T1_sub_ids = [
            'prex006','prex009','prex010','prex011','prex012',
            'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
            'prex027','prex028',
            'prex033','prex037','prex038','prex040',
            'prex041','prex042','prex044','prex045',
            'prex046','prex047','prex048','prex049','prex050','prex052','prex051'
            ]
#去掉了 5和9被
T2M_sub_ids = [
            'prex006M','prex009M','prex010M','prex011M','prex012M',
            'prex015M','prex016M','prex017M','prex018M','prex019M','prex022M','prex023M',
            'prex027M','prex028M',
            'prex033M','prex037M','prex038M','prex040S',
             'prex041M','prex042M','prex044M','prex045M'
             ,'prex046M','prex047M','prex048M','prex049S','prex050M','prex052M','prex051M'

            ]

T2S_sub_ids = [
            'prex006S','prex009S','prex010S','prex011S','prex012S',
            'prex015S','prex016S','prex017S','prex018S','prex019S','prex022S','prex023S',
            'prex027S','prex028S'
            ,'prex033S','prex037S','prex038S','prex040M'
            ,'prex041S','prex042S','prex044S','prex045S'
            ,'prex046S','prex047S','prex048S','prex049M','prex050S','prex052S','prex051S'
            ]


#
# T1_sub_ids = [
#
#                         'prex015S','prex016S','prex018S','prex019S','prex022S','prex023S',
#                         'prex027S','prex028S'
#                         ,'prex033S','prex037S','prex038S'
#                         ,'prex042S','prex044S','prex045S','prex046S','prex047S','prex048S','prex049S','prex052S']
# #去掉了 5和9被试
# T2M_sub_ids = [
#
#             'prex015M','prex016M','prex018M','prex019M','prex022M','prex023M',
#             'prex027M','prex028M'
#             ,'prex033M','prex037M','prex038M'
#              ,'prex042M','prex044M','prex045M','prex046M','prex047M','prex048M','prex049M','prex052M']
#
# T2S_sub_ids = [
#
#                         'prex015','prex016','prex018','prex019','prex022','prex023',
#                         'prex027','prex028'
#                         ,'prex033','prex037','prex038',
#                         'prex042','prex044','prex045','prex046','prex047','prex048','prex049','prex052'
#             ]

###不管三七二一，全部被试放进去

## 这里是  T2S-TMS    VS     T1S-TMS
# T2M_sub_ids = [
#             'prex006','prex009','prex010','prex011','prex012',
#             'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
#             'prex026','prex027','prex028',
#             'prex033','prex037','prex038','prex040',
#             'prex041','prex042','prex044','prex045',
#             'prex046','prex047','prex048','prex049','prex050','prex052',
#
#             ]
# #去掉了 5和9被试
# T1_sub_ids = [
#             'prex006M','prex009M','prex010M','prex011M','prex012M',
#             'prex015M','prex016M','prex017M','prex018M','prex019M','prex022M','prex023M',
#             'prex026M','prex027M','prex028M'
#             ,'prex033M','prex037M','prex038M','prex040S'
#              ,'prex041M','prex042M','prex044M','prex045M'
#              ,'prex046M','prex047M','prex048M','prex049S','prex050M','prex052M',
#
#             ]
#
# T2S_sub_ids = [
#             'prex006S','prex009S','prex010S','prex011S','prex012S',
#             'prex015S','prex016S','prex017S','prex018S','prex019S','prex022S','prex023S',
#             'prex026S','prex027S','prex028S'
#             ,'prex033S','prex037S','prex038S','prex040M'
#             ,'prex041S','prex042S','prex044S','prex045S'
#             ,'prex046S','prex047S','prex048S','prex049M','prex050S','prex052S',
#
#             ]
# # shuffle之前的数据
# T1_sub_ids = [
#             'prex006','prex009','prex010','prex011','prex012',
#             'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
#             'prex026','prex027','prex028'
#             ]
# #去掉了 5和9被试
# T2M_sub_ids = [
#             'prex006M','prex009M','prex010M','prex011M','prex012M',
#             'prex015M','prex016M','prex017M','prex018M','prex019M','prex022M','prex023M',
#             'prex026M','prex027M','prex028M'
#             ]
#
# T2S_sub_ids = [
#             'prex006S','prex009S','prex010S','prex011S','prex012S',
#             'prex015S','prex016S','prex017S','prex018S','prex019S','prex022S','prex023S',
#             'prex026S','prex027S','prex028S'
#
#             ]
#
# T1_sub_ids = [
#     'prex006','prex009','prex010']
# T2M_sub_ids = [
#     'prex006M','prex009M','prex010M']
# T2S_sub_ids = [
#     'prex006S','prex009S','prex010S']

data_path = 'E:\Python LYW\RSA\spaced_learning\pre10\\analysis\\data\\7evoked_clean_equal\\'



list_evoked_T1S = list()
list_evoked_T2S = list()
list_evoked_T2M = list()

for sub_id in T1_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T1S.append(evoked)

for sub_id in T2S_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T2S.append(evoked)

for sub_id in T2M_sub_ids:
    fname = data_path + sub_id + '-ave.fif'
    evoked = mne.read_evokeds(fname)[0]
    list_evoked_T2M.append(evoked)

Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked_clean_equal\prex006-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (101)
        0 CTF compensation matrices available
        nave = 46 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.2, 0] sec)
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked_clean_equal\prex009-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms (101)
        0 CTF compensation matrices available
        nave = 39 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-0.2, 0] sec)
Reading E:\Python LYW\RSA\spaced_learning\pre10\analysis\data\7evoked_clean_equal\prex010-ave.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.0

In [8]:
# evoked.plot_sensors(show_names=True)

In [9]:

# 刷新一下这两个东西
#'用来存放dissimilarity（T1分散 VS T2分散） '
LIST_spaced = []
#'用来存放dissimilarity（T1分散 VS T2集中） '
LIST_massed = []

#时间长。。
#每个field给13个field
#你的时间范围应该是从-0.2到0.8
field_dict = {
        #去掉AFz POz FCz CPz
        #本质只是缺了一个AFz，先用别的替代着？
        "channel_field1": ['Fp1', 'Fp2', 'F3',
                           'F4', 'C3', 'C4', 'P3', 'P4',
                           'O1', 'O2', 'F7', 'F8', 'T7',
                           'T8', 'P7', 'P8', 'Fz', 'Cz',
                           'Pz', 'FC1', 'FC2', 'CP1', 'CP2',
                           'FC5', 'FC6', 'CP5', 'CP6', 'FT9',
                           'FT10', 'TP9', 'TP10', 'F1',
                           'F2', 'C1', 'C2', 'P1', 'P2',
                           'AF3', 'AF4', 'FC3', 'FC4',
                           'CP3', 'CP4', 'PO3', 'PO4',
                           'F5', 'F6', 'C5', 'C6', 'P5',
                           'P6', 'AF7', 'AF8', 'FT7', 'FT8',
                           'TP7', 'TP8', 'PO7', 'PO8', 'Fpz',
                           'CPz', 'POz', 'Oz'],

    }

# field_dict = {
#         #去掉AFz POz FCz CPz
#         #本质只是缺了一个AFz，先用别的替代着？
#         "channel_field1": ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'CPz', 'POz', 'Oz'],
#
#     }
# 如果我只选用顶叶附近的点--3 4区域
# field_dict = {
#         #去掉AFz POz FCz CPz
#         #本质只是缺了一个AFz，先用别的替代着？
#         "channel_field1": ['FT7', 'FC5', 'FC3', 'FC1',
#                            'C5', 'C3', 'C1', 'Cz',
#                            'CP5', 'CP3', 'CP1','TP7','T7',
#
#                            'FT8', 'FC6', 'FC4', 'FC2',
#                            'C6', 'C4', 'C2',
#                            'CP6', 'CP4', 'CP2','TP8','T8',
#
#                            'POz','CPz']
#
#     }

# 这里是为了测试MNE官方的数据
# field_dict = {
#         #去掉AFz POz FCz CPz
#         #本质只是缺了一个AFz，先用别的替代着？
#         "channel_field1":['FP1', 'F3', 'F7', 'FC3', 'C3', 'C5', 'P3', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Oz', 'Pz', 'CPz', 'FP2', 'Fz', 'F4', 'F8', 'FC4', 'FCz', 'Cz', 'C4', 'C6', 'P4', 'P8', 'P10', 'PO8', 'PO4', 'O2']
# }

for i in range(len(T1_sub_ids)):
    # 取出i被试的evoked数据
    T1S = list_evoked_T1S[i]
    T2S = list_evoked_T2S[i]
    T2M = list_evoked_T2M[i]
    #每个被试都要生成两个rdm,这里就是一维数据
    STPSrdm_T1S_T2S = np.zeros([1, 450], dtype=np.float32)
    #range(1)就是只有序列0而已的。
    # 这里的m是指m个field,全脑就相当于只有1个field
    for m in range(1):
        channel_fieldn = field_dict["channel_field" + str(m + 1)]
        eegdataOfOnefield = eegdataOfOneField(channel_fieldn,T1S, T2S)
        #算出一个filed(多个channel组成的pattern)的TPS(时间 模式 相似性)
        OneLineOf_STPS = OneLineOfSTPS(eegdataOfOnefield)
        STPSrdm_T1S_T2S[m, :] = OneLineOf_STPS
    LIST_spaced.append(STPSrdm_T1S_T2S)

    STPSrdm_T1S_T2M = np.zeros([1, 450], dtype=np.float32)
    for m in range(1):
        channel_fieldn = field_dict["channel_field" + str(m + 1)]
        eegdataOfOnefield = eegdataOfOneField(channel_fieldn, T1S, T2M)
        OneLineOf_STPS = OneLineOfSTPS(eegdataOfOnefield)
        STPSrdm_T1S_T2M[m, :] = OneLineOf_STPS
    LIST_massed.append(STPSrdm_T1S_T2M)


Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!

Computing RDM

RDM computing finished!


In [10]:
#先做单个位置的数据提取，然后t检验，然后再循环每个位置，做两重循环了
#所有被试，某行某列。t值，p值。。一个是显著性，一个是大小。

nps = np.zeros([1, 450, 2])
for m in range(1):
    for n in range(450):
        k1 = []
        #被试循环在最内层 k1的长度应该是被试的个数.
        for i in range(len(T2S_sub_ids)):
            #地i个被试的，第m个field的，第n个时间窗
            k1.append(LIST_spaced[i][m][n])
        k2 = []
        for i2 in range(len(T2M_sub_ids)):
            k2.append(LIST_massed[i2][m][n])
        # (static,p1) = stats.levene(k1,k2)
        # nps[m, n] = stats.ttest_ind(k1, k2)
        # 对dissimilarity进行了统计封装。
        # 组间 t检验.
        t, p = stats.ttest_rel(k1, k2)
        print(t)
        nps[m, n, 0] = t
        nps[m, n, 1] = p
        print("nps:"+str(n))
        #打印对照一下，不打印就自动变成0 1？这是pycharm的bug？
        print(nps[m, n])

#去掉含有baseline部分的时间窗
nps2 = np.zeros([1, 350, 2])#设定数据格式，700个单元
nps2 = nps[:, 100:450, :]#选取数据范围

-2.079009987595162
nps:0
[-2.07900999  0.04689368]
-2.041036700106253
nps:1
[-2.0410367   0.05077478]
-1.9999332803515268
nps:2
[-1.99993328  0.05529285]
-1.950792238424456
nps:3
[-1.95079224  0.06115475]
-1.8892864840063843
nps:4
[-1.88928648  0.06925031]
-1.812878987531412
nps:5
[-1.81287899  0.0805855 ]
-1.7217198873471573
nps:6
[-1.72171989  0.09615229]
-1.6182630391470654
nps:7
[-1.61826304  0.11681722]
-1.506312176773019
nps:8
[-1.50631218  0.14318671]
-1.3904809927024508
nps:9
[-1.39048099  0.17533712]
-1.2757039089795825
nps:10
[-1.27570391  0.21254241]
-1.1660302725085645
nps:11
[-1.16603027  0.25343868]
-1.0630434165458795
nps:12
[-1.06304342  0.29684498]
-0.9652826056255768
nps:13
[-0.96528261  0.34266698]
-0.8694659182412847
nps:14
[-0.86946592  0.39198489]
-0.7726653481027455
nps:15
[-0.77266535  0.44619413]
-0.6735798645446699
nps:16
[-0.67357986  0.50610105]
-0.5721363900871467
nps:17
[-0.57213639  0.57179498]
-0.46830043117604586
nps:18
[-0.46830043  0.64319492]
-0.3613

In [11]:
'''
输出nps2?给其他代码使用
'''

# import pandas as pd
#
# # 假设nps是(1,350,2)的ndarray
# data = nps.reshape(-1, 2) # 转换为(350, 2)的二维数组
# df = pd.DataFrame(data, columns=['data1', 'data2']) # 创建DataFrame
# df.to_excel('output.xlsx', index=False) # 保存为Excel文件

'\n输出nps2?给其他代码使用\n'

In [12]:
plt.clf()

# def plot_stats_hotmap(stats, chllabels='', p_threshold=0.05, time_unit=[0, 0.02], lim=[-7, 7], smooth=False, figsize=None, cmap=None, outline=False, threshold=5):
#     """
#     plot the hotmap of statistical results for channels/regions by time sequence
#
#     stats : array
#         The statistical results.
#         The shape of stats must be [n_chls, ts, 2]. n_chls represents the number of channels or regions.
#         ts represents the number of time-points. 2 represents a t-value and a p-value.
#     chllabel : string-array or string-list or None. Default is None.
#         The label for channels/regions.
#         If label=None, the labels will be '1st', '2nd', '3th', '4th', ... automatically.
#     time_unit : array or list [start_t, t_step]. Default is [0, 0.1]
#         The time information of corrs for plotting
#         start_t represents the start time and t_step represents the time between two adjacent time-points. Default
#         time_unit=[0, 0.1], which means the start time of corrs is 0 sec and the time step is 0.1 sec.
#     lim : array or list [min, max]. Default is [-7, -7].
#         The corrs view lims.
#     smooth : boolean True or False
#         Smooth the results or not.
#     figsize : array or list, [size_X, size_Y]
#         The size of the figure.
#         If figsize=None, the size of the figure will be ajusted automatically.
#     cmap : matplotlib colormap or None. Default is None.
#         The colormap for the figure.
#         If cmap=None, the ccolormap will be 'bwr'.
#     outline : bool True or False. Default is False.
#         Outline the significant areas or not.
#     threshold: int. Default is 5.
#         The time threshold for outline.
#         If threshold=5, the time threshold is a window of 5 time-points for each channel/region.
#     """
#
#     # get the number of channels
#     nchls = stats.shape[0]
#
#     # get the number of time-points
#     ts = stats.shape[1]
#
#     # get the start time and the time step
#     start_t = time_unit[0]
#     tstep = time_unit[1]
#
#     # calculate the end time
#     end_t = start_t + ts * tstep
#
#     print(start_t, tstep, end_t)
#
#     # initialize the x
#     x = np.arange(start_t, end_t, tstep)
#
#     # set labels of the channels
#     if chllabels == None:
#
#         chllabels = []
#         for i in range(nchls):
#
#             if i % 10 == 0 and i != 10:
#                 newlabel = str(i + 1) + "st"
#             elif i % 10 == 1 and i != 11:
#                 newlabel = str(i + 1) + "nd"
#             elif i % 10 == 2 and i != 12:
#                 newlabel = str(i + 1) + "rd"
#             else:
#                 newlabel = str(i + 1) + "th"
#             chllabels.append(newlabel)
#
#     if smooth == True:
#
#         t = ts * 50
#
#         x_soft = np.linspace(x.min(), x.max(), t)
#         y_soft = np.zeros([nchls, t])
#
#         samplerate = int(1 / tstep) * 50
#         b, a = signal.butter(4, 2*30/samplerate, 'lowpass')
#
#         for i in range(nchls):
#             f = interp1d(x, stats[i, :, 0], kind='cubic')
#             y_soft[i] = f(x_soft)
#             y_soft[i] = signal.filtfilt(b, a, y_soft[i])
#
#         rlts = y_soft
#
#     if smooth == False:
#         rlts = stats[:, :, 0]
#
#     print(rlts.shape)
#
#     # get min of lims & max of lims
#     limmin = lim[0]
#     limmax = lim[1]
#
#     if outline == True:
#         ps = stats[:, :, 1]
#         tvalues = stats[:, :, 0]
#
#         for i in range(nchls):
#             for j in range(ts):
#
#                 if ps[i, j] < p_threshold and tvalues[i, j] > 0:
#                     ps[i, j] = 1
#                 elif ps[i, j] < p_threshold and tvalues[i, j] < 0:
#                     ps[i, j] = -1
#                 else:
#                     ps[i, j] = 0
#
#         for i in range(nchls):
#             pid = set(())
#             for j in range(ts):
#                 if ps[i, j] != 0:
#                     pid.add(j)
#             pid_list = list(pid)
#             pid_list.sort()
#             pid_set = set()
#             for j in pid_list:
#                 index = 0
#                 for k in range(threshold):
#                     if j+k in pid_list:
#                         index = index
#                     else:
#                         index = index + 1
#                 if index == 0:
#                     for k in range(threshold):
#                         pid_set.add(j+k)
#             pid_list = list(pid_set)
#             pid_list.sort()
#             for j in range(ts):
#                 index = j in pid_list
#                 if index is False:
#                     ps[i, j] = 0
#
#         newps = np.zeros([nchls+2, ts+2], dtype=np.float)
#         newps[1:nchls+1, 1:ts+1] = ps
#
#         x = np.linspace(start_t-0.5*tstep, end_t+0.5*tstep, ts+2)
#         y = np.linspace(-0.08, 0.16*nchls+0.08, nchls+2)
#         X, Y = np.meshgrid(x, y)
#         plt.contour(X, Y, newps, (-0.5, 0.5), linewidths=3)
#
#     fig = plt.gcf()
#     size = fig.get_size_inches()
#
#     if figsize == None:
#         size_x = ts*tstep*(size[0]-2)+2
#         size_y = nchls*0.2*(size[1]-1.5)+1.5
#     else:
#         size_x = figsize[0]
#         size_y = figsize[1]
#
#     fig.set_size_inches(size_x, size_y)
#
#     if cmap == None:
#         plt.imshow(rlts, extent=(start_t, end_t, 0, nchls*0.16), clim=(limmin, limmax), origin='lower')
#     else:
#         plt.imshow(rlts, extent=(start_t, end_t, 0, nchls * 0.16), clim=(limmin, limmax), origin='lower', cmap=cmap)
#
#     cb = plt.colorbar()
#     cb.ax.tick_params(labelsize=16)
#     font = {'size': 18}
#     cb.set_label("Dissimilarity", fontdict=font)
#
#     xi = []
#
#     for i in range(nchls):
#         xi.append(0.16*i + 0.08)
#
#     yi = chllabels
#
#     plt.tick_params(labelsize=18)
#     plt.yticks(xi, yi, fontsize=18)
#
#     plt.ylabel("wholebrain", fontsize=20)
#     plt.xlabel("Time (s)", fontsize=20)
#
#     plt.show()
#
# #调用，设定threshold=5
# plot_stats_hotmap(nps2, time_unit=[0, 0.002],lim=[-2,2], smooth=True, threshold=5 ,outline=True, cmap = "pink" )
def plot_stats_hotmap(stats, chllabels='', p_threshold=0.01, time_unit=[0, 0.02], lim=[-7, 7], smooth=False, figsize=None, cmap=None, outline=False, threshold=5 ):
    """
    plot the hotmap of statistical results for channels/regions by time sequence

    stats : array
        The statistical results.
        The shape of stats must be [n_chls, ts, 2]. n_chls represents the number of channels or regions.
        ts represents the number of time-points. 2 represents a t-value and a p-value.
    chllabel : string-array or string-list or None. Default is None.
        The label for channels/regions.
        If label=None, the labels will be '1st', '2nd', '3th', '4th', ... automatically.
    time_unit : array or list [start_t, t_step]. Default is [0, 0.1]
        The time information of corrs for plotting
        start_t represents the start time and t_step represents the time between two adjacent time-points. Default
        time_unit=[0, 0.1], which means the start time of corrs is 0 sec and the time step is 0.1 sec.
    lim : array or list [min, max]. Default is [-7, -7].
        The corrs view lims.
    smooth : boolean True or False
        Smooth the results or not.
    figsize : array or list, [size_X, size_Y]
        The size of the figure.
        If figsize=None, the size of the figure will be ajusted automatically.
    cmap : matplotlib colormap or None. Default is None.
        The colormap for the figure.
        If cmap=None, the ccolormap will be 'bwr'.
    outline : bool True or False. Default is False.
        Outline the significant areas or not.
    threshold: int. Default is 5.
        The time threshold for outline.
        If threshold=5, the time threshold is a window of 5 time-points for each channel/region.
    """

    # get the number of channels
    nchls = stats.shape[0]

    # get the number of time-points
    ts = stats.shape[1]

    # get the start time and the time step
    start_t = time_unit[0]
    tstep = time_unit[1]

    # calculate the end time
    end_t = start_t + ts * tstep

    print(start_t, tstep, end_t)

    # initialize the x
    x = np.arange(start_t, end_t, tstep)

    # set labels of the channels
    if chllabels == None:

        chllabels = []
        for i in range(nchls):

            if i % 10 == 0 and i != 10:
                newlabel = str(i + 1) + "st"
            elif i % 10 == 1 and i != 11:
                newlabel = str(i + 1) + "nd"
            elif i % 10 == 2 and i != 12:
                newlabel = str(i + 1) + "rd"
            else:
                newlabel = str(i + 1) + "th"
            chllabels.append(newlabel)

    if smooth == True:

        t = ts * 50

        x_soft = np.linspace(x.min(), x.max(), t)
        y_soft = np.zeros([nchls, t])

        samplerate = int(1 / tstep) * 50
        b, a = signal.butter(4, 2*30/samplerate, 'lowpass')

        for i in range(nchls):
            f = interp1d(x, stats[i, :, 0], kind='cubic')
            y_soft[i] = f(x_soft)
            y_soft[i] = signal.filtfilt(b, a, y_soft[i])

        rlts = y_soft

    if smooth == False:
        rlts = stats[:, :, 0]

    print(rlts.shape)

    # get min of lims & max of lims
    limmin = lim[0]
    limmax = lim[1]

    if outline == True:
        ps = stats[:, :, 1]
        tvalues = stats[:, :, 0]

        for i in range(nchls):
            for j in range(ts):

                if ps[i, j] < p_threshold and tvalues[i, j] > 0:
                    ps[i, j] = 1
                elif ps[i, j] < p_threshold and tvalues[i, j] < 0:
                    ps[i, j] = -1
                else:
                    ps[i, j] = 0

        for i in range(nchls):
            pid = set(())
            for j in range(ts):
                if ps[i, j] != 0:
                    pid.add(j)
            pid_list = list(pid)
            pid_list.sort()
            pid_set = set()
            for j in pid_list:
                index = 0
                for k in range(threshold):
                    if j+k in pid_list:
                        index = index
                    else:
                        index = index + 1
                if index == 0:
                    for k in range(threshold):
                        pid_set.add(j+k)
            pid_list = list(pid_set)
            pid_list.sort()
            for j in range(ts):
                index = j in pid_list
                if index is False:
                    ps[i, j] = 0

        newps = np.zeros([nchls+2, ts+2], dtype=np.float)
        newps[1:nchls+1, 1:ts+1] = ps

        x = np.linspace(start_t-0.5*tstep, end_t+0.5*tstep, ts+2)
        y = np.linspace(-0.08, 0.16*nchls+0.08, nchls+2)
        X, Y = np.meshgrid(x, y)
        plt.contour(X, Y, newps, (-0.5, 0.5), linewidths=3)

    fig = plt.gcf()
    size = fig.get_size_inches()

    if figsize == None:
        size_x = ts*tstep*(size[0]-2)+2
        size_y = nchls*0.2*(size[1]-1.5)+1.5
    else:
        size_x = figsize[0]
        size_y = figsize[1]

    fig.set_size_inches(size_x, size_y)

    if cmap == None:
        plt.imshow(rlts, extent=(start_t, end_t, 0, nchls*0.16), clim=(limmin, limmax), origin='lower')
    else:
        plt.imshow(rlts, extent=(start_t, end_t, 0, nchls * 0.16), clim=(limmin, limmax), origin='lower', cmap=cmap)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=16)
    font = {'size': 18}
    cb.set_label("Dissimilarity", fontdict=font)

    xi = []

    for i in range(nchls):
        xi.append(0.16*i + 0.08)

    yi = chllabels

    plt.tick_params(labelsize=18)
    plt.yticks(xi, yi, fontsize=18)

    plt.ylabel("wholebrain", fontsize=20)
    plt.xlabel("Time (s)", fontsize=20)

    plt.show()
plot_stats_hotmap(nps2,time_unit=[0, 0.002] , smooth=True, threshold=5, outline=True,
                  p_threshold=0.05, lim=[-0.5,3], chllabels=channel_fieldn, figsize=[20,30], cmap='bwr')



0 0.002 0.7000000000000001
(1, 17500)


C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_15232\220626919.py:312: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  newps = np.zeros([nchls+2, ts+2], dtype=np.float)


ValueError: The number of FixedLocator locations (1), usually from a call to set_ticks, does not match the number of labels (63).

In [ ]:
#如果想要画出线性的图呢？另一种呈现方式？但是你对比的是哪两个啊？画出来的是t值的均值？
#把求t改成求平均值就能得到你想要的两个  一维 数据了。
#分别装到两个 一维 向量里面

# 一维数据存储均值就行
#1是T1 vs T2S

ave1 = np.zeros([1, 350])
ave2 = np.zeros([1, 350])
#m是channel
# channel是什么啊这里，
for m_channel in range(1):
    #保留基线的
    # for n_timepoint in range(0,450):
    # 这里也是去掉了基线的。
    for n_timepoint in range(100,450):
        k1 = []
        for i in range(len(T2S_sub_ids)):
            #从rdm里面取出想要的差异值
            #固定了channel和时间点，所有被试的
            k1.append(LIST_spaced[i][m_channel][n_timepoint])
        k2 = []
        for i in range(len(T2M_sub_ids)):
            k2.append(LIST_massed[i][m_channel][n_timepoint])
        # (static,p1) = stats.levene(k1,k2)
        # nps[m, n] = stats.ttest_ind(k1, k2)
        # nps[m, n] = stats.ttest_rel(k1, k2)
        #n-100是因为。。前面的200ms不要，因为包括了基线部分的数据。其实你要愿意，从0.2开始都行？
        ave1[m_channel, n_timepoint-100]= np.mean(k1)
        ave2[m_channel, n_timepoint-100]= np.mean(k2)


#作图
#这里是差异值，dissimilarity
# 单个被试分析的时候就可以考虑用这个来看看，因为不能进行统计。
plt.clf()
x = range(len(ave1))
x = np.arange(0.1,0.70,0.002)

plt.plot(x,ave1[0,50:], 'b')
plt.plot(x,ave2[0,50:], 'r')

plt.show()


In [ ]:
'''
直接画 nps2 的看看 nps2 是对similarity做统计。
画什么呢？画p值，还是画t值
'''

x = np.linspace(0, 0.7, 350)
plt.plot(x, nps2[0,:,0])
plt.show()

In [ ]:
x = np.linspace(-0.2, 0.7, 450)
plt.plot(x, nps[0,:,1])

np.save('./nps2.npy',nps2)

In [ ]:
#和上面无关，临时用来输出channels给使用。

save_file = open('./channels.txt', 'w')
for channel_name in evoked.ch_names:
    save_file.write(channel_name)
    save_file.write('\n')
save_file.close()


In [ ]:
#如果想要画出线性的图呢？另一种呈现方式？但是你对比的是哪两个啊？画出来的是t值的均值？
#把求t改成求平均值就能得到你想要的两个  一维 数据了。
#分别装到两个 一维 向量里面

# 一维数据存储均值就行
#1是T1 vs T2S
# ave1 = np.zeros([1, 450])
# ave2 = np.zeros([1, 450])
# #m是channel
# # channel是什么啊这里，
# for m_channel in range(1):
#     #保留基线的
#     for n_timepoint in range(0,450):
#     # 这里也是去掉了基线的。
#     # for n_timepoint in range(100,450):
#         k1 = []
#         for i in range(len(T2S_sub_ids)):
#             #从rdm里面取出想要的差异值
#             #固定了channel和时间点，所有被试的
#             k1.append(LIST_spaced[i][m_channel][n_timepoint])
#         k2 = []
#         for i in range(len(T2M_sub_ids)):
#             k2.append(LIST_massed[i][m_channel][n_timepoint])
#         # (static,p1) = stats.levene(k1,k2)
#         # nps[m, n] = stats.ttest_ind(k1, k2)
#         # nps[m, n] = stats.ttest_rel(k1, k2)
#         #n-100是因为。。
#         ave1[m_channel, n_timepoint]= np.mean(k1)
#         ave2[m_channel, n_timepoint]= np.mean(k2)
# #作图
# #这里是差异值，dissimilarity
# # 单个被试分析的时候就可以考虑用这个来看看，因为不能进行统计。
# plt.clf()
# x = range(len(ave1))
# x = np.arange(-0.1,0.80,0.002)
#
# plt.plot(x,ave1[0,:], 'b')
# plt.plot(x,ave2[0,:], 'r')
# plt.show()



In [14]:
from neurora.rsa_plot import plot_tbytsim_withstats

# 这里是similarity的画图
array_massed = np.array(LIST_massed)
array_spaced = np.array(LIST_spaced)

#結構需要修正一下
# 換位置+降維度
array_massed = array_massed.reshape([29,450])
array_spaced = array_spaced.reshape([29,450])

np.save('./dissimilarity_massed',array_massed)
np.save('./dissimilarity_spaced',array_spaced)
# array_massed = array_massed[:,100:450]
# array_spaced = array_spaced[:,100:450]

In [24]:
# 为何是450啊 0.9呢，
# 为何这里配合的是 -0.2 到0.7而不是-0.1到0.8  因为是和真实情况保持一致。
# 做出来的图应该是缺了一块的，一开始应该接近1才对,不要被误导了。
plot_tbytsim_withstats(array_massed,start_time=-0.2,end_time=0.7,time_interval=0.002,smooth=True,
                       color='r',p=0, avgshow=True, xlim=[-0.2, 0.7], fontsize=20)

plot_tbytsim_withstats(array_spaced,start_time=-0.2,end_time=0.7,time_interval=0.002,smooth=True,
                       color='b',p=0, avgshow=True,xlim=[-0.2, 0.7], fontsize=20)

#问题是你要把显著区域，根据t和p加上啊，还需要校验一下？


Significant time-windows:

Significant time-windows:


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [27]:
'''
https://github.com/ZitongLu1996/NeuroRA/blob/master/neurora/rsa_plot.py
把函数用到的别的包也导入

'''

import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import signal
from scipy.stats import ttest_1samp, ttest_rel

from neurora.stuff import get_affine, get_bg_ch2, get_bg_ch2bet, correct_by_threshold, \
    clusterbased_permutation_1d_1samp_1sided, clusterbased_permutation_2d_1samp_1sided, \
    clusterbased_permutation_1d_1samp_2sided, clusterbased_permutation_2d_2sided, smooth_1d
from decimal import Decimal

def plot_tbyt_diff_decoding_acc(acc1, acc2, start_time=0, end_time=1, time_interval=0.01, chance=0.5, p=0.05, cbpt=True,
                                clusterp=0.05, stats_time=[0, 1], color1='r', color2='b', label1='Condition1',
                                label2='Condition2', xlim=[0, 1], ylim=[0.4, 0.8], xlabel='Time (s)',
                                ylabel='Decoding Accuracy', figsize=[6.4, 3.6], x0=0, ticksize=12, fontsize=16,
                                markersize=2, legend_fontsize=14, title=None, title_fontsize=16, avgshow=False):

    """
    Plot the differences of time-by-time decoding accuracies between two conditions
    Parameters
    ----------
    acc1 : array
        The decoding accuracies under condition1.
        The size of acc1 should be [n_subs, n_ts]. n_subs, n_ts represent the number of subjects and number of
        time-points.
    acc2 : array
        The decoding accuracies under condition2.
        The size of acc2 should be [n_subs, n_ts]. n_subs, n_ts represent the number of subjects and number of
        time-points.
    start_time : int or float. Default is 0.
        The start time.
    end_time : int or float. Default is 1.
        The end time.
    time_interval : float. Default is 0.01.
        The time interval between two time samples.
    chance : float. Default is 0.5.
        The chance level.
    p : float. Default is 0.05.
        The threshold of p-values.
    cbpt : bool True or False. Default is True.
        Conduct cluster-based permutation test or not.
    clusterp : float. Default is 0.05.
        The threshold of cluster-defining p-values.
    stats_time : array or list [stats_time1, stats_time2]. Default os [0, 1].
        Time period for statistical analysis.
    color1 : matplotlib color or None. Default is 'r'.
        The color for the curve under condition1.
    color2 : matplotlib color or None. Default is 'r'.
        The color for the curve under condition2.
    label1 : string-array. Default is 'Condition1'.
        The Label of acc1's condition.
    label2 : string-array. Default is 'Condition2'.
        The Label of acc2's condition.
    xlim : array or list [xmin, xmax]. Default is [0, 1].
        The x-axis (time) view lims.
    ylim : array or list [ymin, ymax]. Default is [0.4, 0.8].
        The y-axis (decoding accuracy) view lims.
    xlabel : string. Default is 'Time (s)'.
        The label of x-axis.
    ylabel : string. Default is 'Decoding Accuracy'.
        The label of y-axis.
    figsize : array or list, [size_X, size_Y]. Default is [6.4, 3.6].
        The size of the figure.
    x0 : float. Default is 0.
        The Y-axis is at x=x0.
    ticksize : int or float. Default is 12.
        The size of the ticks.
    fontsize : int or float. Default is 16.
        The fontsize of the labels.
    markersize : int or float. Default is 2.
        The size of significant marker.
    legend_fontsize : int or float. Default is 14.
        The fontsize of the legend.
    title : string-array. Default is None.
        The title of the figure.
    title_fontsize : int or float. Default is 16.
        The fontsize of the title.
    avgshow : boolen True or False. Default is False.
        Show the averaging decoding accuracies or not.
    """

    if len(np.shape(acc1)) != 2 or len(np.shape(acc2)) != 2:

        return "Invalid input!"

    nsubs, nts = np.shape(acc1)
    tstep = float(Decimal((end_time - start_time) / nts).quantize(Decimal(str(time_interval))))

    if tstep != time_interval:

        return "Invalid input!"

    delta1 = (stats_time[0] - start_time) / tstep - int((stats_time[0] - start_time) / tstep)
    delta2 = (stats_time[1] - start_time) / tstep - int((stats_time[1] - start_time) / tstep)
    if delta1 == 0:
        stats_time1 = int((stats_time[0] - start_time) / tstep)
    else:
        stats_time1 = int((stats_time[0] - start_time) / tstep) + 1
    if delta2 == 0:
        stats_time2 = int((stats_time[1] - start_time) / tstep)
    else:
        stats_time2 = int((stats_time[1] - start_time) / tstep) + 1

    yminlim = ylim[0]
    ymaxlim = ylim[1]

    avg1 = np.average(acc1, axis=0)
    err1 = np.zeros([nts])
    for t in range(nts):
        err1[t] = np.std(acc1[:, t], ddof=1) / np.sqrt(nsubs)

    avg2 = np.average(acc2, axis=0)
    err2 = np.zeros([nts])
    for t in range(nts):
        err2[t] = np.std(acc2[:, t], ddof=1) / np.sqrt(nsubs)

    if cbpt == True:

        ps1_stats = clusterbased_permutation_1d_1samp_1sided(acc1[:, stats_time1:stats_time2], level=chance,
                                                             p_threshold=p, clusterp_threshold=clusterp, iter=1000)
        ps1 = np.zeros([nts])
        ps1[stats_time1:stats_time2] = ps1_stats
        ps2_stats = clusterbased_permutation_1d_1samp_1sided(acc2[:, stats_time1:stats_time2], level=chance,
                                                             p_threshold=p, clusterp_threshold=clusterp, iter=1000)
        ps2 = np.zeros([nts])
        ps2[stats_time1:stats_time2] = ps2_stats
        ps_stats = clusterbased_permutation_1d_1samp_2sided(acc1[:, stats_time1:stats_time2]-
                                                            acc2[:, stats_time1:stats_time2], level=0, p_threshold=p,
                                                            clusterp_threshold=clusterp, iter=1000)
        ps = np.zeros([nts])
        ps[stats_time1:stats_time2] = ps_stats

    else:
        ps1 = np.zeros([nts])
        ps2 = np.zeros([nts])
        ps = np.zeros([nts])
        for t in range(nts):
            if t >= stats_time1 and t< stats_time2:
                ps1[t] = ttest_1samp(acc1[:, t], chance, alternative="greater")[1]
                ps2[t] = ttest_1samp(acc2[:, t], chance, alternative="greater")[1]
                if ps1[t] < p:
                    ps1[t] = 1
                else:
                    ps1[t] = 0
                if ps2[t] < p:
                    ps2[t] = 1
                else:
                    ps2[t] = 0
                if ttest_rel(acc1[:, t], acc2[:, t], alternative="greater")[1] < p/2:
                    ps[t] = 1
                elif ttest_rel(acc1[:, t], acc2[:, t], alternative="less")[1] < p/2:
                    ps[t] = -1
                else:
                    ps[t] = 0

    print('\nSignificant time-windows for condition 1:')
    for t in range(nts):
        if t == 0 and ps1[t] == 1:
            print(str(int(start_time * 1000)) + 'ms to ', end='')
        if t > 0 and ps1[t] == 1 and ps1[t - 1] == 0:
            print(str(int((start_time + t * tstep) * 1000)) + 'ms to ', end='')
        if t < nts - 1 and ps1[t] == 1 and ps1[t + 1] == 0:
            print(str(int((start_time + (t + 1) * tstep) * 1000)) + 'ms')
        if t == nts - 1 and ps1[t] == 1:
            print(str(int(stats_time[1] * 1000)) + 'ms')

    print('\nSignificant time-windows for condition 2:')
    for t in range(nts):
        if t == 0 and ps2[t] == 1:
            print(str(int(start_time * 1000)) + 'ms to ', end='')
        if t > 0 and ps2[t] == 1 and ps2[t - 1] == 0:
            print(str(int((start_time + t * tstep) * 1000)) + 'ms to ', end='')
        if t < nts - 1 and ps2[t] == 1 and ps2[t + 1] == 0:
            print(str(int((start_time + (t + 1) * tstep) * 1000)) + 'ms')
        if t == nts - 1 and ps2[t] == 1:
            print(str(int(stats_time[1] * 1000)) + 'ms')

    print('\nSignificant time-windows for condition 1 > condition 2:')
    for t in range(nts):
        if t == 0 and ps[t] == 1:
            print(str(int(start_time * 1000)) + 'ms to ', end='')
        if t > 0 and ps[t] == 1 and ps2[t - 1] < 1:
            print(str(int((start_time + t * tstep) * 1000)) + 'ms to ', end='')
        if t < nts - 1 and ps[t] == 1 and ps[t + 1] < 1:
            print(str(int((start_time + (t + 1) * tstep) * 1000)) + 'ms')
        if t == nts - 1 and ps[t] == 1:
            print(str(int(stats_time[1] * 1000)) + 'ms')

    print('\nSignificant time-windows for condition 2 > condition 1:')
    for t in range(nts):
        if t == 0 and ps[t] == -1:
            print(str(int(start_time * 1000)) + 'ms to ', end='')
        if t > 0 and ps[t] == -1 and ps2[t - 1] > -1:
            print(str(int((start_time + t * tstep) * 1000)) + 'ms to ', end='')
        if t < nts - 1 and ps[t] == -1 and ps[t + 1] > -1:
            print(str(int((start_time + (t + 1) * tstep) * 1000)) + 'ms')
        if t == nts - 1 and ps[t] == -1:
            print(str(int(stats_time[1] * 1000)) + 'ms')

    for t in range(nts):
        if ps1[t] == 1:
            plt.plot(t*tstep+start_time+0.5*tstep, (ymaxlim-yminlim)*0.95+yminlim, 's', color=color1, alpha=0.8,
                     markersize=markersize)
        if ps2[t] == 1:
            plt.plot(t*tstep+start_time+0.5*tstep, (ymaxlim-yminlim)*0.91+yminlim, 's', color=color2, alpha=0.8,
                     markersize=markersize)
        if ps[t] == 1:
            xi = [t*tstep+start_time, t*tstep+tstep+start_time]
            ymin = [avg2[t] + err2[t]]
            ymax = [avg1[t] - err1[t]]
            plt.fill_between(xi, ymax, ymin, facecolor="grey", alpha=0.2)
        if ps[t] == -1:
            xi = [t*tstep+start_time, t*tstep+tstep+start_time]
            ymin = [avg1[t] + err1[t]]
            ymax = [avg2[t] - err2[t]]
            plt.fill_between(xi, -10, 10, facecolor="grey", alpha=0.2)

    fig = plt.gcf()
    fig.set_size_inches(figsize[0], figsize[1])
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(3)
    ax.spines["left"].set_position(("data", x0))
    ax.spines["bottom"].set_linewidth(3)
    ax.spines["bottom"].set_position(("data", chance))
    x = np.arange(start_time+0.5*tstep, end_time+0.5*tstep, tstep)
    if avgshow is True:
        plt.plot(x, avg1, color=color1, alpha=0.95)
        plt.plot(x, avg2, color=color2, alpha=0.95)
    plt.fill_between(x, avg1+err1, avg1-err1, facecolor=color1, alpha=0.75, label=label1)
    plt.fill_between(x, avg2+err2, avg2-err2, facecolor=color2, alpha=0.75, label=label2)
    # plt.fill_between(x, -10, 10, facecolor='grey', alpha=0.2, label=label2)
    plt.ylim(yminlim, ymaxlim)
    plt.xlim(xlim[0], xlim[1])
    plt.tick_params(labelsize=ticksize)
    plt.xlabel(xlabel, fontsize=fontsize)
    plt.ylabel(ylabel, fontsize=fontsize)
    plt.legend()
    ax = plt.gca()
    leg = ax.get_legend()
    ltext = leg.get_texts()
    plt.setp(ltext, fontsize=legend_fontsize)
    plt.title(title, fontsize=title_fontsize)
    plt.show()

    return ps1, ps2, ps


' a function for plotting cross-temporal decoding accuracies '
ps1, ps2, ps =plot_tbyt_diff_decoding_acc(array_spaced,array_massed,start_time=-0.2,end_time=0.7, p=0.05, clusterp=0.05,
                             time_interval=0.002, ylim=[0, 1],
                            stats_time=[0, 0.74],xlim=[-0.2, 0.7])


Permutation test

Side 1 begin:
Calculating: [====================================================================================================] 100.00%
Side 1 finished!


Significant time-windows for condition 1:

Significant time-windows for condition 2:

Significant time-windows for condition 1 > condition 2:
328ms to 330ms to 332ms to 334ms to 336ms to 338ms to 340ms to 342ms to 344ms to 346ms to 348ms to 350ms to 352ms to 354ms to 356ms to 358ms to 360ms to 362ms to 364ms to 366ms to 368ms to 370ms to 372ms to 374ms to 376ms to 377ms to 379ms to 381ms to 383ms to 385ms to 387ms to 389ms to 391ms to 393ms to 395ms to 397ms to 399ms to 401ms to 403ms to 406ms to 408ms to 410ms to 412ms to 414ms to 416ms to 418ms to 420ms to 422ms to 424ms to 426ms to 428ms to 430ms to 432ms to 434ms to 436ms to 438ms to 440ms to 442ms to 444ms

Significant time-windows for condition 2 > condition 1:


In [ ]:
import matplotlib.pyplot as plt

nps2 = np.zeros([1, 350, 2])
x = np.linspace(0,350,351)#
y = np.linspace(0,1,10000)
plt.contourf(x,y,nps2[0, :, 1])



In [64]:

ps = plot_tbytsim_withstats(nps2,start_time=0,end_time=0.7,time_interval=0.002,smooth=True,
                       color='r',p=0.05, avgshow=True, xlim=[0, 1], ylim=[-1, 3],fontsize=20)


Significant time-windows:


C:\Users\SilverBullet\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\SilverBullet\AppData\Roaming\Python\Python39\site-packages\neurora\stuff.py:1126: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  ts[t], p = ttest_1samp(results[:, t], level, alternative='greater')


In [28]:
significant_indices = []
for i in range(array_massed.shape[1]):
    t_stat, p_val = ttest_rel(array_spaced[:, i], array_massed[:, i])
    if p_val < 0.05:
        significant_indices.append(i)